In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import os
import re
import time

from gensim.models import Word2Vec
from tqdm import tqdm

tqdm.pandas()

In [0]:
df_train = pd.read_csv("/content/gdrive/My Drive/comp/train.csv")
df_test = pd.read_csv("/content/gdrive/My Drive/comp/test.csv")

import string
import re

word_dict = dict()

def add_word_dict(y):
  for w in y.split():
    if not w in word_dict:
      word_dict[w] = 1
    else:
      word_dict[w] += 1

def remove_row(y):
  processed = re.sub('[^a-zA-Z ]', '', y)
  y = change_punc(y)
  temp = []
  for x in y.split():
    x = remove_digit(x)
    x = reduce_lengthening(x)
    if len(x) > 1: 
      temp.append(x)
  clean = ' '.join(temp)
  clean = re.sub(' {2,}', ' ', clean)
  return(clean.strip())

def remove_digit(w):
  result = ''.join([i for i in w if not i.isdigit()])
  return result
#   table = result.maketrans({key: None for key in string.punctuation})
#   result = result.translate(table)
def change_punc(result):
  clean = re.sub(r"[,.;:@#?!%&$+-]+\ *", " ", result)
  return clean
def reduce_lengthening(text):
  pattern = re.compile(r"(.)\1{2,}")
  return pattern.sub(r"\1\1", text)

In [0]:
#data cleaning for each row
df_train["title"] = df_train["title"].apply(remove_row)
df_test["title"] = df_test["title"].apply(remove_row)

In [8]:
sentences = pd.concat([df_train['title'], df_test['title']],axis=0)
train_sentences = list(sentences.progress_apply(str.split).values)

100%|██████████| 839017/839017 [00:03<00:00, 262015.92it/s]


In [11]:
all_words = set()

def word_set(y):
  for word in y.split():
    if word not in all_words:
      all_words.add(word)

sentences.apply(word_set)
print(len(all_words))

51360


In [0]:
model = Word2Vec(sentences=train_sentences, 
                 sg=1, 
                 size=100,  
                 workers=4,iter=10)

In [0]:
model.wv.save_word2vec_format('custom_glove_100d.txt')

In [14]:
model.wv.most_similar('aace')

KeyError: ignored